In [1]:
import pickle
import time
from typing import List
import datetime as dt

import numpy as np

import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_z0_1q, get_z1_1q, get_x0_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.projected_linear_estimator import ProjectedLinearEstimator

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from quara.settings import Settings

In [4]:
def estimate(name: str, true_object: State, num_data: List[int], iterations: int, on_para_eq_constraint: bool=True):
    qst = StandardQst(povms, on_para_eq_constraint=on_para_eq_constraint)

    # generate empi dists and calc estimate
    obj_sequences = []
    start = time.time()
    for iteration in tqdm(range(iterations)):
        try:
            empi_dists_seq = qst.generate_empi_dists_sequence(
            true_object, num_data
        )

            estimator = ProjectedLinearEstimator()
            obj_sequence = estimator.calc_estimate_sequence(qst, empi_dists_seq)
            obj_sequences.append(obj_sequence.estimated_qoperation_sequence)
        except ValueError as e:
            print("[Error]")
            print(e)
            print(f"empi_dists_seq: {empi_dists_seq}")
        
    end = time.time()
    print(f"time(s)={end - start}")

    identify = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # path = f"qst_data/{name}_{identify}.pkl"
    # with open(path, "wb") as f:
    #     pickle.dump(obj_sequences, f)
    
    return obj_sequences

# True Object = State([1/np.sqrt(2), 0, 0, 1/np.sqrt(2)])

In [11]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
povms = [povm_x, povm_y, povm_z]

In [12]:
true_object = get_z0_1q(c_sys)
num_data = [1000]
iterations = 100

param_affine_est_linear = estimate("z0_affine", true_object, num_data, iterations, on_para_eq_constraint=True)
param_linear_est_linear = estimate("z0_linear", true_object, num_data, iterations, on_para_eq_constraint=False)

param_affine_est_linear = [p[0] for p in param_affine_est_linear]
param_linear_est_linear = [p[0] for p in param_linear_est_linear]

  0%|          | 0/100 [00:00<?, ?it/s]/Users/tomoko/project/rcast/workspace/quara/quara/objects/state.py:277: ComplexWarning:

Casting complex values to real discards the imaginary part

  4%|▍         | 4/100 [00:00<00:02, 37.68it/s]

time(s)=2.5890769958496094


100%|██████████| 100/100 [00:02<00:00, 40.30it/s]

time(s)=2.4859020709991455


## on_para_eq_constraint = True

In [13]:
unphysical_qobjects = [qobject for qobject in param_affine_est_linear if not qobject.is_physical()]
len(unphysical_qobjects)

100

In [14]:
print(len([qo for qo in unphysical_qobjects if qo.is_positive_semidefinite()]))

print(len([qo for qo in unphysical_qobjects if qo.is_trace_one()]))

0
100


In [15]:
violation_result = physicality_violation_check.get_physicality_violation_result_for_state(param_affine_est_linear)
violation_result.keys()

dict_keys(['violation_list'])

In [16]:
physicality_violation_check.make_prob_dist_histogram(violation_result["violation_list"], bin_size=0.01)

## on_para_eq_constraint = False

In [17]:
unphysical_qobjects = [qobject for qobject in param_linear_est_linear if not qobject.is_physical()]
len(unphysical_qobjects)

100

In [18]:
print(len([qo for qo in unphysical_qobjects if qo.is_positive_semidefinite()]))

print(len([qo for qo in unphysical_qobjects if qo.is_trace_one()]))

0
100


In [19]:
violation_result = physicality_violation_check.get_physicality_violation_result_for_state(param_linear_est_linear)
violation_result.keys()

100%|██████████| 100/100 [00:00<00:00, 6871.40it/s]


dict_keys(['sorted_eigenvalues_list', 'sum_of_eigenvalues'])

In [20]:
true_eigs = sorted([eig.real for eig in true_object.calc_eigenvalues()], reverse=True)

for i, values in enumerate(violation_result["sorted_eigenvalues_list"]):
    fig = physicality_violation_check.make_prob_dist_histogram(values, bin_size=0.0001)
    fig.update_layout(title=f"N={num_data[0]}, Nrep={len(values)}")
    fig.update_xaxes( title=f"Eigenvalue (i={i})")
    
    x_value = true_eigs[i]
    fig.add_shape(
                type="line",
                line_color="red",
                line_width=2,
                opacity=0.5,
                x0=x_value,
                x1=x_value,
                xref="x",
                y0=0,
                y1=1,
                yref="paper",
            )
    fig.show()

In [21]:
value_list = violation_result["sum_of_eigenvalues"]["less_than_zero"]
fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.0001, annotation_vlines=[0])

# TODO: iterationsとnum_dataを与える方法
n_rep = len(violation_result["sorted_eigenvalues_list"][0])
n = num_data[0]
title = f"N={n}, Nrep={n_rep}, Number of Unphysical estimates={len(value_list)}"
fig.update_layout(title=title)
fig.update_xaxes( title=f"Sum of unphysical eigenvalues (<0)")
fig.show()

value_list = violation_result["sum_of_eigenvalues"]["greater_than_one"]
fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.0001, annotation_vlines=[1])

n_rep = len(violation_result["sorted_eigenvalues_list"][0])
n = num_data[0]
title = f"N={n}, Nrep={n_rep}, Number of Unphysical estimates={len(value_list)}"
fig.update_layout(title=title)  # TODO
fig.update_xaxes( title=f"Sum of unphysical eigenvalues (>1)")
fig.show()

# True Object = State([ 1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6) ])

In [56]:
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

vec = np.array([1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)],dtype=np.float64)
true_object = State(c_sys, vec)
num_data = [1000]  # N
iterations = 100  # Nrep

In [57]:
povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
povms = [povm_x, povm_y, povm_z]

In [58]:
name = "case_2"
# param_affine_est_linear = estimate(name, true_object, num_data, iterations, on_para_eq_constraint=True)
param_linear_est_linear = estimate(name, true_object, num_data, iterations, on_para_eq_constraint=False)

# param_affine_est_linear = [p[0] for p in param_affine_est_linear]
param_linear_est_linear = [p[0] for p in param_linear_est_linear]

100%|██████████| 100/100 [00:02<00:00, 45.68it/s]

time(s)=2.197434902191162


## on_para_eq_constraint = True

In [59]:
unphysical_qobjects = [qobject for qobject in param_affine_est_linear if not qobject.is_physical()]
len(unphysical_qobjects)

0

In [60]:
print(len([qo for qo in unphysical_qobjects if qo.is_positive_semidefinite()]))

print(len([qo for qo in unphysical_qobjects if qo.is_trace_one()]))

0
0


In [61]:
violation_result = physicality_violation_check.get_physicality_violation_result_for_state(param_affine_est_linear)
violation_result.keys()

dict_keys(['violation_list'])

In [62]:
value_list = violation_result["violation_list"]
physicality_violation_check.make_prob_dist_histogram(value_list, x_range=(-1, 2), bin_size=0.01)

## on_para_eq_constraint = False

In [63]:
unphysical_qobjects = [qobject for qobject in param_linear_est_linear if not qobject.is_physical()]
len(unphysical_qobjects)

34

In [64]:
print(len([qo for qo in unphysical_qobjects if qo.is_positive_semidefinite()]))

print(len([qo for qo in unphysical_qobjects if qo.is_trace_one()]))

0
34


In [65]:
violation_result = physicality_violation_check.get_physicality_violation_result_for_state(param_linear_est_linear)
violation_result.keys()

100%|██████████| 100/100 [00:00<00:00, 4993.75it/s]


dict_keys(['sorted_eigenvalues_list', 'sum_of_eigenvalues'])

In [32]:
true_eigs = sorted([eig.real for eig in true_object.calc_eigenvalues()], reverse=True)

for i, values in enumerate(violation_result["sorted_eigenvalues_list"]):
    fig = physicality_violation_check.make_prob_dist_histogram(values, bin_size=0.0001)
    fig.update_layout(title=f"Nrep={len(values)}")
    fig.update_xaxes( title=f"Eigenvalue (i={i})")
    
    x_value = true_eigs[i]
    fig.add_shape(
                type="line",
                line_color="red",
                line_width=2,
                opacity=0.5,
                x0=x_value,
                x1=x_value,
                xref="x",
                y0=0,
                y1=1,
                yref="paper",
            )
    fig.show()

In [66]:
value_list = violation_result["sum_of_eigenvalues"]["less_than_zero"]
value_list

[-2.0367752404834896e-13,
 -1.703841836757399e-13,
 -3.4843228253676004e-13,
 -2.447322929268413e-13,
 -1.7797222880819619e-12,
 -1.0131484581534071e-13,
 -2.471886110728618e-13,
 -2.0404522122055634e-13,
 -7.629234193772171e-11,
 -1.265394784040632e-13,
 -2.8953173100098675e-13,
 -1.0538138697651806e-13,
 -2.1519099603047857e-12,
 -2.8050153811782243e-13,
 -1.6019942933215253e-13,
 -1.8142617902653193e-13,
 -1.0868953870991856e-13,
 -5.376551888665212e-13,
 -3.5756617378247e-13,
 -6.311102379082488e-13,
 -4.5358753842817816e-13,
 -1.0976736753213498e-13,
 -2.0749092629693046e-13,
 -1.7469414979048305e-13,
 -2.498520991041332e-13,
 -4.077221474417983e-13,
 -1.4225699853666398e-13,
 -1.7201497582167644e-13,
 -8.894456571503775e-13,
 -1.1033674696295056e-12,
 -1.2824579417207499e-12,
 -4.075150883727794e-13,
 -1.4078986140842822e-13,
 -1.5264771081210015e-13]

In [35]:
value_list = violation_result["sum_of_eigenvalues"]["less_than_zero"]
fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.0001, annotation_vlines=[0])

# TODO: iterationsとnum_dataを与える方法
n_rep = len(violation_result["sorted_eigenvalues_list"][0])
n = num_data[0]
title = f"N={n}, Nrep={n_rep}, Number of Unphysical estimates={len(value_list)}"
fig.update_layout(title=title)
fig.update_xaxes( title=f"Sum of unphysical eigenvalues (<0)")
fig.show()

value_list = violation_result["sum_of_eigenvalues"]["greater_than_one"]
fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.0001, annotation_vlines=[1])

n_rep = len(violation_result["sorted_eigenvalues_list"][0])
n = num_data[0]
title = f"N={n}, Nrep={n_rep}, Number of Unphysical estimates={len(value_list)}"
fig.update_layout(title=title)  # TODO
fig.update_xaxes( title=f"Sum of unphysical eigenvalues (>1)")
fig.show()

# True Object = State([1/np.sqrt(2), 0, 0, 0])

In [36]:
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

vec = np.array([1/np.sqrt(2), 0, 0, 0],dtype=np.float64)
true_object = State(c_sys, vec)
num_data = [1000]  # N
iterations = 100  # Nrep

In [37]:
povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
povms = [povm_x, povm_y, povm_z]

In [38]:
name = "case_3"
param_affine_est_linear = estimate(name, true_object, num_data, iterations, on_para_eq_constraint=True)
param_linear_est_linear = estimate(name, true_object, num_data, iterations, on_para_eq_constraint=False)

param_affine_est_linear = [p[0] for p in param_affine_est_linear]
param_linear_est_linear = [p[0] for p in param_linear_est_linear]

  5%|▌         | 5/100 [00:00<00:02, 39.27it/s]

time(s)=1.889470100402832


100%|██████████| 100/100 [00:02<00:00, 37.37it/s]

time(s)=2.6832659244537354


## on_para_eq_constraint = True

In [39]:
unphysical_qobjects = [qobject for qobject in param_affine_est_linear if not qobject.is_physical()]
len(unphysical_qobjects)

0

In [40]:
print(len([qo for qo in unphysical_qobjects if qo.is_positive_semidefinite()]))

print(len([qo for qo in unphysical_qobjects if qo.is_trace_one()]))

0
0


In [41]:
violation_result = physicality_violation_check.get_physicality_violation_result_for_state(param_affine_est_linear)
violation_result.keys()

dict_keys(['violation_list'])

In [42]:
value_list = violation_result["violation_list"]
physicality_violation_check.make_prob_dist_histogram(value_list, x_range=(-1, 2), bin_size=0.01)

## on_para_eq_constraint = False

In [43]:
unphysical_qobjects = [qobject for qobject in param_linear_est_linear if not qobject.is_physical()]
len(unphysical_qobjects)

0

In [46]:
violation_result = physicality_violation_check.get_physicality_violation_result_for_state(param_linear_est_linear)
violation_result.keys()

100%|██████████| 100/100 [00:00<00:00, 5611.03it/s]


dict_keys(['sorted_eigenvalues_list', 'sum_of_eigenvalues'])

In [47]:
true_eigs = sorted([eig.real for eig in true_object.calc_eigenvalues()], reverse=True)

for i, values in enumerate(violation_result["sorted_eigenvalues_list"]):
    fig = physicality_violation_check.make_prob_dist_histogram(values, bin_size=0.0001)
    fig.update_layout(title=f"Nrep={len(values)}")
    fig.update_xaxes( title=f"Eigenvalue (i={i})")
    
    x_value = true_eigs[i]
    fig.add_shape(
                type="line",
                line_color="red",
                line_width=2,
                opacity=0.5,
                x0=x_value,
                x1=x_value,
                xref="x",
                y0=0,
                y1=1,
                yref="paper",
            )
    fig.show()

In [48]:
value_list = violation_result["sum_of_eigenvalues"]["less_than_zero"]
fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.0001, annotation_vlines=[0])

# TODO: iterationsとnum_dataを与える方法
n_rep = len(violation_result["sorted_eigenvalues_list"][0])
n = num_data[0]
title = f"N={n}, Nrep={n_rep}, Number of Unphysical estimates={len(value_list)}"
fig.update_layout(title=title)
fig.update_xaxes( title=f"Sum of unphysical eigenvalues (<0)")
fig.show()

value_list = violation_result["sum_of_eigenvalues"]["greater_than_one"]
fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.0001, annotation_vlines=[1])

n_rep = len(violation_result["sorted_eigenvalues_list"][0])
n = num_data[0]
title = f"N={n}, Nrep={n_rep}, Number of Unphysical estimates={len(value_list)}"
fig.update_layout(title=title)  # TODO
fig.update_xaxes( title=f"Sum of unphysical eigenvalues (>1)")
fig.show()

# Case 4: 

In [49]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
povms = [povm_x, povm_y, povm_z]

In [50]:
vec = np.array([1/np.sqrt(2), 0, 0, 1/np.sqrt(2)], dtype=np.float64)
true_object = State(c_sys, vec)
num_data = [1000]  # N
iterations = 100  # Nrep

name = "case_4"
# param_affine_est_linear = estimate(name, true_object, num_data, iterations, on_para_eq_constraint=True)
param_linear_est_linear = estimate(name, true_object, num_data, iterations, on_para_eq_constraint=False)

# param_affine_est_linear = [p[0] for p in param_affine_est_linear]
param_linear_est_linear = [p[0] for p in param_linear_est_linear]

100%|██████████| 100/100 [00:03<00:00, 32.67it/s]

time(s)=3.0646438598632812


In [51]:
unphysical_qobjects = [qobject for qobject in param_linear_est_linear if not qobject.is_physical()]
len(unphysical_qobjects)

100

In [52]:
value_list = violation_result["sum_of_eigenvalues"]["less_than_zero"]

value_list

[]

In [53]:
violation_result = physicality_violation_check.get_physicality_violation_result_for_state(param_linear_est_linear)
violation_result.keys()

100%|██████████| 100/100 [00:00<00:00, 5279.77it/s]


dict_keys(['sorted_eigenvalues_list', 'sum_of_eigenvalues'])

In [54]:
true_eigs = sorted([eig.real for eig in true_object.calc_eigenvalues()], reverse=True)

for i, values in enumerate(violation_result["sorted_eigenvalues_list"]):
    fig = physicality_violation_check.make_prob_dist_histogram(values, bin_size=0.0001)
    fig.update_layout(title=f"Nrep={len(values)}")
    fig.update_xaxes( title=f"Eigenvalue (i={i})")
    
    x_value = true_eigs[i]
    fig.add_shape(
                type="line",
                line_color="red",
                line_width=2,
                opacity=0.5,
                x0=x_value,
                x1=x_value,
                xref="x",
                y0=0,
                y1=1,
                yref="paper",
            )
    fig.show()

In [55]:
value_list = violation_result["sum_of_eigenvalues"]["less_than_zero"]
fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.0001, annotation_vlines=[0])

# TODO: iterationsとnum_dataを与える方法
n_rep = len(violation_result["sorted_eigenvalues_list"][0])
n = num_data[0]
title = f"N={n}, Nrep={n_rep}, Number of Unphysical estimates={len(value_list)}"
fig.update_layout(title=title)
fig.update_xaxes( title=f"Sum of unphysical eigenvalues (<0)")
fig.show()

value_list = violation_result["sum_of_eigenvalues"]["greater_than_one"]
fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.0001, annotation_vlines=[1])

n_rep = len(violation_result["sorted_eigenvalues_list"][0])
n = num_data[0]
title = f"N={n}, Nrep={n_rep}, Number of Unphysical estimates={len(value_list)}"
fig.update_layout(title=title)  # TODO
fig.update_xaxes( title=f"Sum of unphysical eigenvalues (>1)")
fig.show()